<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chord-Progression-Examples" data-toc-modified-id="Chord-Progression-Examples-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chord Progression Examples</a></span><ul class="toc-item"><li><span><a href="#Method-1:-Building-from-intervals" data-toc-modified-id="Method-1:-Building-from-intervals-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Method 1: Building from intervals</a></span></li><li><span><a href="#Method-2:-Building-randomly-from-scale" data-toc-modified-id="Method-2:-Building-randomly-from-scale-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Method 2: Building randomly from scale</a></span></li></ul></li><li><span><a href="#Rhythm-Examples" data-toc-modified-id="Rhythm-Examples-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Rhythm Examples</a></span><ul class="toc-item"><li><span><a href="#Example-1:-Medium-note-density,-granular-quantization,-varying-note-durations" data-toc-modified-id="Example-1:-Medium-note-density,-granular-quantization,-varying-note-durations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example 1: Medium note density, granular quantization, varying note durations</a></span></li><li><span><a href="#Example-2:-High-note-density,-coarse-quantization,-bar-length-durations" data-toc-modified-id="Example-2:-High-note-density,-coarse-quantization,-bar-length-durations-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Example 2: High note density, coarse quantization, bar-length durations</a></span></li><li><span><a href="#Example-3:-Triplets-Durations-(subtle)" data-toc-modified-id="Example-3:-Triplets-Durations-(subtle)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Example 3: Triplets Durations (subtle)</a></span></li><li><span><a href="#Example-4:-Triplet-Quantization" data-toc-modified-id="Example-4:-Triplet-Quantization-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Example 4: Triplet Quantization</a></span></li><li><span><a href="#Example-5:-Fifth-Quantization" data-toc-modified-id="Example-5:-Fifth-Quantization-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Example 5: Fifth Quantization</a></span></li><li><span><a href="#Example-6:-Seventh-Quantization" data-toc-modified-id="Example-6:-Seventh-Quantization-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Example 6: Seventh Quantization</a></span></li><li><span><a href="#Example-7:-Ninth-Quantization" data-toc-modified-id="Example-7:-Ninth-Quantization-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Example 7: Ninth Quantization</a></span></li><li><span><a href="#Example-8:-Adding-emphasis-randomly" data-toc-modified-id="Example-8:-Adding-emphasis-randomly-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Example 8: Adding emphasis randomly</a></span></li><li><span><a href="#Example-9:-Adding-emphasis-directly" data-toc-modified-id="Example-9:-Adding-emphasis-directly-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Example 9: Adding emphasis directly</a></span></li><li><span><a href="#Example-10:-Adding-emphasis-randomly-2" data-toc-modified-id="Example-10:-Adding-emphasis-randomly-2-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>Example 10: Adding emphasis randomly 2</a></span></li></ul></li><li><span><a href="#Sync-the-chord-progression-with-the-rhythm" data-toc-modified-id="Sync-the-chord-progression-with-the-rhythm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sync the chord progression with the rhythm</a></span></li><li><span><a href="#Finish-and-write-to-midi-file" data-toc-modified-id="Finish-and-write-to-midi-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finish and write to midi file</a></span></li></ul></div>

In [408]:
import os

from midi_skirt import *

In [409]:
pattern = midi.Pattern(resolution=pc.resolution)
track = midi.Track()
pattern.append(track)

In [410]:
bpm = 120
numerator = 7
denominator = 8
track.append(midi.SetTempoEvent(bpm=bpm))
track.append(midi.TimeSignatureEvent(numerator=numerator, denominator=denominator))

In [411]:
pc = PatternConstants(resolution=440, beats_per_bar=numerator*2)

In [412]:
scale.NAMED_SCALES.keys()

['major',
 'wholetone',
 'melodicminor',
 'harmonicminor',
 'wholehalf',
 'bluesminor',
 'chromatic',
 'ionian',
 'augmentedfifth',
 'locrian',
 'oriental',
 'minor',
 'mixolydian',
 'japanese',
 'pentatonicminor',
 'lydian',
 'phrygian',
 'halfwhole',
 'diminished',
 'pentatonicmajor',
 'aeolian',
 'dorian',
 'bluesmajor',
 'augmented']

# Chord Progression Examples

## Method 1: Building from intervals

In [413]:
c7sharp9 = ChordBuilder().build_from_intervals("C", 5, ["1", "3", "5", "b7", "#9"])
am7 = ChordBuilder().build_from_intervals("A", 5, ["1", "b3", "5", "b7"])
g6 = ChordBuilder().build_from_intervals("G", 5, ["1", "3", "5", "6"])
f69 = ChordBuilder().build_from_intervals("F", 5, ["1", "3", "5", "6", "9"])
bb13 = ChordBuilder().build_from_intervals("Bb", 5, ["1", "3", "5", "b7", "9", "13"])

chord_progression = ChordProgression(
    chords=[am7, g6, f69, bb13],
    changes=[pc.bar, pc.bar, pc.bar, pc.bar]
)

chord_progression.repeat_progression(18)

## Method 2: Building randomly from scale

In [414]:
num_chords_and_changes = 64
chord_progression = ChordProgression(
    changes=[pc.bar] * num_chords_and_changes
)

chord_progression.build_progression_randomly_from_scale("G", 4, "locrian", num_chords_and_changes)

# Rhythm Examples

## Example 1: Medium note density, granular quantization, varying note durations

In [415]:
# rhythm specs
rhythm_len = pc.bar * 64
start_tick = 0
quantization = pc.sixteenth_note
note_density = .5
note_len_choices = pc.get_notes_2()

In [416]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 2: High note density, coarse quantization, bar-length durations

In [417]:
# rhythm specs
rhythm_len = pc.bar * 64
start_tick = 0
quantization = pc.beat
note_density = 10.0
note_len_choices = [pc.beat]

In [418]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 3: Triplets Durations (subtle)

In [419]:
# # rhythm specs
# rhythm_len = pc.bar * 64
# start_tick = 0
# quantization = pc.half_note
# note_density = 10.0
# note_len_choices = pc.get_notes_3()

In [420]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 4: Triplet Quantization

In [421]:
# # rhythm specs
# rhythm_len = pc.bar * 64
# start_tick = 0
# quantization = pc.half_triplet
# note_density = 10.0
# note_len_choices = pc.get_notes_3()

In [422]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 5: Fifth Quantization

In [423]:
# # rhythm specs
# rhythm_len = pc.bar * 64
# start_tick = 0
# quantization = pc.half_fifth
# note_density = 10.0
# note_len_choices = pc.get_notes_5()

In [424]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 6: Seventh Quantization

In [425]:
# # rhythm specs
# rhythm_len = pc.bar * 64
# start_tick = 0
# quantization = pc.half_seventh
# note_density = 10.0
# note_len_choices = pc.get_notes_7()

In [426]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 7: Ninth Quantization

In [427]:
# # rhythm specs
# rhythm_len = pc.bar * 64
# start_tick = 0
# quantization = pc.half_ninth
# note_density = 10.0
# note_len_choices = pc.get_notes_9()

In [428]:
rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)

## Example 8: Adding emphasis randomly

In [429]:
def example_8():
    # rhythm specs
    rhythm_len = pc.bar * 64
    start_tick = 0
    quantization = pc.beat
    note_density = 10.0
    note_len_choices = [pc.beat]

    rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
    rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)
    rhythm.build_emphasis_ticks_randomly(emphasis_quantization=pc.beat, container=pc.bar)
    
    return rhythm

## Example 9: Adding emphasis directly

In [430]:
def example_9():
    rhythm_len = pc.bar * 64
    start_tick = 0
    quantization = pc.beat
    note_density = 10.0
    note_len_choices = [pc.beat]

    rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
    rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)
    rhythm.build_emphasis_ticks_directly(emphasis_quantization=pc.beat, container=pc.bar, emphasis_divisions=[2,5])  # sounds nice in 11/4 (3+8)
    return rhythm

## Example 10: Adding emphasis randomly 2

In [431]:
def example_10():
    # rhythm specs
    rhythm_len = pc.bar * 64
    start_tick = 0
    quantization = pc.eighth_note
    note_density = 1
    note_len_choices = [pc.eighth_note, pc.quarter_note]

    rhythm = Rhythm(rhythm_len=rhythm_len, start_tick=start_tick, quantization=quantization)
    rhythm.build_rhythm_randomly(note_density=note_density, note_len_choices=note_len_choices)
    rhythm.build_emphasis_ticks_randomly(emphasis_quantization=pc.eighth_note, container=pc.bar)
    
    return rhythm

In [432]:
rhythm = example_10()

Emphasis on: [15  3 24 11]


# Sync the chord progression with the rhythm

In [433]:
cpr = ChordProgressionRhythm(
    rhythm=rhythm,
    chord_progression=chord_progression,
    tick_method="direct",  # "direct" or "random" or "random_once" or "random_asc" or "random_desc"
    vel_method="random",  # "direct" or "random"
    tick_noise=[-55, 55])  # only used if tick_method is done randomly

all_staged_events = []
for chord in cpr.chords:
    for staged_event in chord.staged_events:
        all_staged_events.append(staged_event)
df = convert_staging_events_to_dataframe(all_staged_events)
df.sort_values(by=["tick", "duration"], inplace=True)

# Finish and write to midi file

In [434]:
# add to track
track = add_tuples_to_track(track, df)

# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=get_max_tick(track) + 2 * pc.whole_note)
track.append(eot)

track = make_ticks_rel(track)

midi.write_midifile("example.mid", pattern)

os.system("timidity --adjust-tempo={} example.mid".format(tempo))

256